Task 3: Does the weather affect the delay?

Use the API to pull the weather information for flights. There is no need to get weather for ALL flights. We can choose the right representative sample. Let's focus on four weather types:

- sunny
- cloudy
- rainy
- snow. 

Test the hypothesis that these 4 delays are from the same distribution. If they are not, which ones are significantly different?

In [2]:
import pandas as pd 
import numpy as np

In [3]:

sample = pd.read_csv('G_PSQL_data/random_sample.txt')

In [4]:
sample.columns 

Index(['fl_date', 'mkt_unique_carrier', 'branded_code_share', 'mkt_carrier',
       'mkt_carrier_fl_num', 'op_unique_carrier', 'tail_num',
       'op_carrier_fl_num', 'origin_airport_id', 'origin', 'origin_city_name',
       'dest_airport_id', 'dest', 'dest_city_name', 'crs_dep_time', 'dep_time',
       'dep_delay', 'taxi_out', 'wheels_off', 'wheels_on', 'taxi_in',
       'crs_arr_time', 'arr_time', 'arr_delay', 'cancelled',
       'cancellation_code', 'diverted', 'dup', 'crs_elapsed_time',
       'actual_elapsed_time', 'air_time', 'flights', 'distance',
       'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay',
       'late_aircraft_delay', 'first_dep_time', 'total_add_gtime',
       'longest_add_gtime', 'no_name'],
      dtype='object')

In [5]:
#use this for final weather merge
sample = sample[["fl_date","origin_city_name","dep_delay"]]

In [6]:
sample['dep_delay'] = sample["dep_delay"].fillna(0)

In [7]:
len(sample)

106443

In [8]:
#use this to merge lat lon data 

sample2 = sample.origin_city_name.unique()



In [9]:
city_name= pd.DataFrame(sample2)

In [10]:
city_name.head()

,0
0,"Boston, MA"
1,"Ponce, PR"
2,"Fort Lauderdale, FL"
3,"San Juan, PR"
4,"Seattle, WA"


In [11]:
city_name[['city','state']] = city_name[0].str.split(',',expand=True)

In [12]:
city_name.drop(columns=[0], inplace=True)

In [13]:
city_name

,city,state
0,Boston,MA
1,Ponce,PR
2,Fort Lauderdale,FL
3,San Juan,PR
4,Seattle,WA
...,...,...
357,Alpena,MI
358,Petersburg,AK
359,St. Cloud,MN
360,Rota,TT


In [154]:
city_db = pd.read_csv('uscities.csv')

In [204]:
city_db.columns

Index(['city', 'city_ascii', 'state_id', 'state_name', 'county_fips',
       'county_name', 'lat', 'lng', 'population', 'density', 'source',
       'military', 'incorporated', 'timezone', 'ranking', 'zips', 'id'],
      dtype='object')

In [220]:
city_LL = city_db[["city","state_id","lat","lng"]].copy()

In [221]:
cols = ['city','state_id']

city_LL['combined'] = city_LL[cols].apply(lambda row: ', '.join(row.values.astype(str)), axis=1)


In [222]:
city_LL

,city,state_id,lat,lng,combined
0,New York,NY,40.6943,-73.9249,"New York, NY"
1,Los Angeles,CA,34.1139,-118.4068,"Los Angeles, CA"
2,Chicago,IL,41.8373,-87.6862,"Chicago, IL"
3,Miami,FL,25.7839,-80.2102,"Miami, FL"
4,Dallas,TX,32.7936,-96.7662,"Dallas, TX"
...,...,...,...,...,...
28333,Gross,NE,42.9461,-98.5697,"Gross, NE"
28334,Lotsee,OK,36.1334,-96.2091,"Lotsee, OK"
28335,The Ranch,MN,47.3198,-95.6952,"The Ranch, MN"
28336,Shamrock,OK,35.9113,-96.5772,"Shamrock, OK"


In [230]:
city_name = city_name.merge(city_LL, how='left', left_on=[0],right_on=['combined'])

In [236]:
city_name.drop(columns=["city","state_id","combined"],inplace=True)

In [238]:
city_name.to_csv('city_name.csv', columns=[0,'lat','lng'], header=False, index=False)
    

In [246]:
weather = pd.read_csv("weather_data.csv")


In [260]:
weather.shape

(208874, 3)

In [250]:
sample_weather = sample.merge(weather, how='left', left_on=["fl_date", 'origin_city_name'], right_on=['datetime','name'])

In [261]:
sample_weather.isnull().sum()

fl_date                 0
origin_city_name        0
dep_delay               0
name                22740
datetime            22740
conditions          22740
dtype: int64

In [262]:
weather.conditions.unique()

array(['Clear', 'Snow', 'Partially cloudy', 'Snow, Partially cloudy',
       'Snow, Overcast', 'Rain, Overcast', 'Rain, Partially cloudy',
       'Rain', 'Overcast', nan, 'Rain, Fog'], dtype=object)

In [266]:
weather = weather.replace({'Partially cloudy': 'Cloudy',
                           'Overcast': 'Cloudy',
                           'Snow, Overcast': 'Snow',
                           'Rain, Overcast': 'Rain',
                          'Snow, Partially cloudy': 'Snow',
                          'Rain, Partially cloudy': 'Rain',
                          'Rain, Fog': 'Rain'})




In [ ]:
weather.to_csv('weather_final.csv')

In [16]:
weather2019 = pd.read_csv("G_PSQL_data/weather2019.csv")

In [22]:
weather2019 = weather2019.replace({'Partially cloudy': 'Cloudy',
                           'Overcast': 'Cloudy',
                           'Snow, Overcast': 'Snow',
                           'Rain, Overcast': 'Rain',
                          'Snow, Partially cloudy': 'Snow',
                          'Rain, Partially cloudy': 'Rain',
                          'Rain, Fog': 'Rain'})

In [21]:
weather2019

,name,datetime,conditions
0,"Waterloo, IA",2019-12-14,Partially cloudy
1,"Waterloo, IA",2019-12-15,Partially cloudy
2,"Waterloo, IA",2019-12-16,"Snow, Partially cloudy"
3,"Waterloo, IA",2019-12-17,Partially cloudy
4,"Waterloo, IA",2019-12-18,Clear
...,...,...,...
6511,"Rochester, NY",2019-12-27,"Rain, Partially cloudy"
6512,"Rochester, NY",2019-12-28,Partially cloudy
6513,"Rochester, NY",2019-12-29,"Rain, Partially cloudy"
6514,"Rochester, NY",2019-12-30,"Rain, Partially cloudy"


In [ ]:
weather = weather.replace({'Partially cloudy': 'Cloudy',
                           'Overcast': 'Cloudy',
                           'Snow, Overcast': 'Snow',
                           'Rain, Overcast': 'Rain',
                          'Snow, Partially cloudy': 'Snow',
                          'Rain, Partially cloudy': 'Rain',
                          'Rain, Fog': 'Rain'})

In [24]:
#list of weather to replace
db2_weather = ['Clear','Cloudy','Rain','Snow']
db1_weather = [0,1,2,3]

#run it thru 
weather2019["conditions"] = weather2019["conditions"].replace(to_replace=db2_weather,value=db1_weather)

In [26]:
weather2019.to_csv("weather_dec2019.csv",index=False)